In [ ]:
import numpy as np
import tensorflow as tf

# reload model
model_status = tf.keras.models.load_model('model_status')
# "model_status" is a folder, and it should be the same place as the app.py
model_status.summary() # Check architecture, can be omitted when deployed

# reload the second model
model_months = tf.keras.models.load_model('model_months')
# "model_months" is a folder, and it should be the same place as the app.py
model_months.summary() # Check architecture, can be omitted when deployed

In [ ]:
def onehot(y, K):
    """
    One-hot encoding function.
    Inputs
    - y: (uint8) a numpy array of shape (N, 1) containing labels; y[i] = k means 
        that X[i] has label k, where 0 <= k < K.
    - K: total number of classes
    Returns a numpy array
    - y_onehot: (float) the encoded labels of shape (N, K)
    """
    N = y.shape[0] # get input size
    y_onehot = np.zeros((N, K)) # initialize onehot array
    for i in range(N):
        y_onehot[i, y[i].astype(int)] = 1 # put a "1" in its place (y[i]) as label
    return y_onehot

### Input shape for classifier

These 12 features can be used directly.

    CODE_GENDER                1.0
    FLAG_OWN_CAR               1.0
    FLAG_OWN_REALTY             1.0
    CNT_CHILDREN               0.0
    AMT_INCOME_TOTAL            427500.0
    DAYS_BIRTH                -12005.0
    DAYS_EMPLOYED              -4542.0
    FLAG_MOBIL                1.0
    FLAG_WORK_PHONE             1.0
    FLAG_PHONE                0.0
    FLAG_EMAIL                0.0
    CNT_FAM_MEMBERS             2.0
These 5 features need one hot encoding.

    INCOME_TYPE(OH).0           0.0
    INCOME_TYPE(OH).1           0.0
    INCOME_TYPE(OH).2           0.0
    INCOME_TYPE(OH).3           0.0
    INCOME_TYPE(OH).4           1.0
        HOUSING_TYPE(OH).0            0.0
        HOUSING_TYPE(OH).1            0.0
        HOUSING_TYPE(OH).2            0.0
        HOUSING_TYPE(OH).3            0.0
        HOUSING_TYPE(OH).4            1.0
        HOUSING_TYPE(OH).5            0.0
            EDUCATION_TYPE(OH).0          0.0
            EDUCATION_TYPE(OH).1          1.0
            EDUCATION_TYPE(OH).2          0.0
            EDUCATION_TYPE(OH).3          0.0
            EDUCATION_TYPE(OH).4          0.0
                FAMILY_STATUS(OH).0            1.0
                FAMILY_STATUS(OH).1            0.0
                FAMILY_STATUS(OH).2            0.0
                FAMILY_STATUS(OH).3            0.0
                FAMILY_STATUS(OH).4            0.0
                    OCCUPATION_TYPE(OH).0          1.0
                    OCCUPATION_TYPE(OH).1          0.0
                    OCCUPATION_TYPE(OH).2          0.0
                    OCCUPATION_TYPE(OH).3          0.0
                    OCCUPATION_TYPE(OH).4          0.0
                    OCCUPATION_TYPE(OH).5          0.0
                    OCCUPATION_TYPE(OH).6          0.0
                    OCCUPATION_TYPE(OH).7          0.0
                    OCCUPATION_TYPE(OH).8          0.0
                    OCCUPATION_TYPE(OH).9          0.0
                    OCCUPATION_TYPE(OH).10         0.0
                    OCCUPATION_TYPE(OH).11         0.0
                    OCCUPATION_TYPE(OH).12         0.0
                    OCCUPATION_TYPE(OH).13         0.0
                    OCCUPATION_TYPE(OH).14         0.0
                    OCCUPATION_TYPE(OH).15         0.0
                    OCCUPATION_TYPE(OH).16         0.0
                    OCCUPATION_TYPE(OH).17         0.0
                    OCCUPATION_TYPE(OH).18         0.0

In [ ]:
def sigmoid(x):
    y = 1/(1 + np.exp(-x))
    return y

In [ ]:
def activation(y):
    """
    modded sigmoid function
    Input
    - y: (float) output of the classifier, shape 1X2
        [1, 0] => no overdue
        [0, 1] => potential overdue
    Return
    - pred: (int) prediction, scalar
        0 => potential credit card
        1 => no credit card
    - poss: (float) possibility of getting a credit card
    """
    if (y[0,0] >= y[0,1]):
        pred = 0; # give prediction
        poss = sigmoid(y[0, 0]*2);
    else:
        pred = 1; # give prediction
        poss = 1 - sigmoid(y[0, 1]*2);
    return (pred, poss)

### activation Example

In [ ]:
P_status = activation(model_status.predict(input_encoded.reshape(1, 52)))[1]
P_months = activation(model_months.predict(input_encoded.reshape(1, 52)))[1]
P = (P_status + P_months)/2 # average